In [1]:
data<-read.csv("/home/iiitb/DataAnalyticsProject/Data/big_student_clear_third_version.csv", header=TRUE)

In [2]:
class(data$institute)

[1] "factor"

In [12]:
library(dplyr)
library(rpart)
library(ggplot2)
library(caret)


In [4]:
head(data)

X,institute,course_id,year,semester,userid_DI,viewed,explored,certified,final_cc_cname_DI,⋯,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,incomplete_flag,age
4,HarvardX,PH207x,2012,Fall,MHxPC130313697,0,0,0,India,⋯,0,2012-07-24,2013-07-27,6,3,197757,0,0,0,23
6,HarvardX,PH207x,2012,Fall,MHxPC130237753,1,0,0,United States,⋯,0,2012-07-24,2012-12-24,107,8,7,2,0,0,19
7,HarvardX,CS50x,2012,Summer,MHxPC130202970,1,0,0,United States,⋯,0,2012-07-24,2013-03-28,8,1,197757,1,0,0,24
20,HarvardX,CS50x,2012,Summer,MHxPC130223941,1,0,0,Other Middle East/Central Asia,⋯,0,2012-07-24,2013-07-15,25,2,197757,4,0,0,20
22,HarvardX,PH207x,2012,Fall,MHxPC130317399,0,0,0,Australia,⋯,0,2012-07-24,2012-08-25,3,2,197757,0,0,0,32
23,HarvardX,CS50x,2012,Summer,MHxPC130191782,1,0,0,Pakistan,⋯,0,2012-07-24,2013-05-05,2,2,197757,1,0,0,22


In [6]:
data_filtered<-select(data,institute,semester,viewed,explored,certified,final_cc_cname_DI,LoE_DI,gender,
                      grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts,incomplete_flag
                     ,age)

In [7]:
set.seed(101) 
# Now Selecting 70% of data as sample from total 'n' rows of the data  
sample <- sample.int(n = nrow(data_filtered), size = floor(.70*nrow(data)), replace = F)
train <- data[sample, ]
test  <- data[-sample, ]
nrow(train)
nrow(test)

[1] 291844

[1] 125077

In [19]:
attach(data_filtered)
dtfit <- rpart(institute~course_id+semester+final_cc_cname_DI+LoE_DI+gender+ age,data= train,method="class")
dtfit

The following objects are masked from data_filtered (pos = 3):

    age, certified, explored, final_cc_cname_DI, gender, grade,
    incomplete_flag, institute, last_event_DI, LoE_DI, nchapters,
    ndays_act, nevents, nforum_posts, nplay_video, semester,
    start_time_DI, viewed

The following objects are masked from data_filtered (pos = 6):

    age, certified, explored, final_cc_cname_DI, gender, grade,
    incomplete_flag, institute, last_event_DI, LoE_DI, nchapters,
    ndays_act, nevents, nforum_posts, nplay_video, semester,
    start_time_DI, viewed



n= 291844 

node), split, n, loss, yval, (yprob)
      * denotes terminal node

1) root 291844 123901 MITx (0.4245453 0.5754547)  
  2) course_id=CB22x,CS50x,ER22x,PH207x,PH278x 123901      0 HarvardX (1.0000000 0.0000000) *
  3) course_id=14.73x,2.01x,3.091x,6.002x,6.00x,7.00x,8.02x,8.MReV 167943      0 MITx (0.0000000 1.0000000) *

In [20]:
summary(dtfit)

Call:
rpart(formula = institute ~ course_id + semester + final_cc_cname_DI + 
    LoE_DI + gender + age, data = train, method = "class")
  n= 291844 

    CP nsplit rel error xerror        xstd
1 1.00      0         1      1 0.002155104
2 0.01      1         0      0 0.000000000

Variable importance
        course_id            gender               age final_cc_cname_DI 
               80                 9                 4                 3 
         semester            LoE_DI 
                2                 2 

Node number 1: 291844 observations,    complexity param=1
  predicted class=MITx      expected loss=0.4245453  P(node) =1
    class counts: 123901 167943
   probabilities: 0.425 0.575 
  left son=2 (123901 obs) right son=3 (167943 obs)
  Primary splits:
      course_id         splits as  RRRRRRRRLLLLL, improve=142598.800, (0 missing)
      gender            splits as  RLRL, improve=  5727.400, (0 missing)
      age               < 25.5 to the right, improve=  4404.934, (0 m

In [21]:
predicted<-predict(dtfit,test,type="class")

In [23]:
confusionMatrix(data=predicted, reference=test$institute)

Warning message in ni[1:m] * nj[1:m]:
“NAs produced by integer overflow”

Confusion Matrix and Statistics

          Reference
Prediction HarvardX  MITx
  HarvardX    52901     0
  MITx            0 72176
                                   
               Accuracy : 1        
                 95% CI : (1, 1)   
    No Information Rate : 0.5771   
    P-Value [Acc > NIR] : < 2.2e-16
                                   
                  Kappa : NA       
 Mcnemar's Test P-Value : NA       
                                   
            Sensitivity : 1.0000   
            Specificity : 1.0000   
         Pos Pred Value : 1.0000   
         Neg Pred Value : 1.0000   
             Prevalence : 0.4229   
         Detection Rate : 0.4229   
   Detection Prevalence : 0.4229   
      Balanced Accuracy : 1.0000   
                                   
       'Positive' Class : HarvardX 
                                   

In [24]:
dtfit2 <- rpart(institute~semester+final_cc_cname_DI+LoE_DI+gender+ age,data= train,method="class")
predicted2<-predict(dtfit2,test,type="class")
confusionMatrix(data=predicted2, reference=test$institute)

Warning message in ni[1:m] * nj[1:m]:
“NAs produced by integer overflow”

Confusion Matrix and Statistics

          Reference
Prediction HarvardX  MITx
  HarvardX    19573 10150
  MITx        33328 62026
                                         
               Accuracy : 0.6524         
                 95% CI : (0.6497, 0.655)
    No Information Rate : 0.5771         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : NA             
 Mcnemar's Test P-Value : < 2.2e-16      
                                         
            Sensitivity : 0.3700         
            Specificity : 0.8594         
         Pos Pred Value : 0.6585         
         Neg Pred Value : 0.6505         
             Prevalence : 0.4229         
         Detection Rate : 0.1565         
   Detection Prevalence : 0.2376         
      Balanced Accuracy : 0.6147         
                                         
       'Positive' Class : HarvardX       
                                         